# EDA of the gex/adt dataset
Work in progress

In [2]:
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

from lab_scripts.utils import utils
utils.change_directory_to_repo()

from lab_scripts.data import dataloader

In [3]:
data = dataloader.load_data('pm/official/gex_to_adt')

# Gex

In [5]:
# Load data
gex = data['train_mod1']
gex_meta = ad.read_h5ad("data/official/common/openproblems_bmmc_cite_phase1/openproblems_bmmc_cite_phase1.manual_formatting.output_rna.h5ad")
gex_meta = gex_meta[gex_meta.obs['is_train'],:]

In [4]:
gex.obs['batch'].value_counts()

s3d6    9977
s2d1    9353
s2d4    5026
s1d1    4721
s1d2       0
s3d7       0
Name: batch, dtype: int64

In [5]:
gex_meta.obs['batch'].value_counts()

s3d6    9977
s2d1    9353
s2d4    5026
s1d1    4721
Name: batch, dtype: int64

In [6]:
# We see that gex_meta is the same as gex, so we will analyse the latter only
del gex

In [7]:
gex_meta.obs.describe()

,n_genes_by_counts,pct_counts_mt,size_factors,pseudotime_order_GEX,pseudotime_order_ADT
count,29077.000000,29077.000000,29077.000000,5791.000000,5739.000000
mean,1440.599993,6.663049,0.948834,0.738605,0.773514
std,847.892033,3.497144,0.850345,0.299480,0.272619
min,103.000000,0.000000,0.002663,0.000000,0.000000
25%,838.000000,4.545455,0.459884,0.737430,0.673726
50%,1257.000000,6.495110,0.710780,0.865717,0.923920
75%,1819.000000,8.584656,1.148076,0.939360,0.966033
max,5815.000000,19.988226,7.837794,1.000000,1.000000


In [11]:
gex_meta.obs['cell_type'].unique()

['Naive CD20+ B IGKC+', 'CD14+ Mono', 'HSC', 'Reticulocyte', 'Normoblast', ..., 'Plasma cell IGKC+', 'Plasma cell IGKC-', 'CD8+ T CD57+ CD45RO+', 'ILC', 'dnT']
Length: 40
Categories (40, object): ['B1 B IGKC+', 'B1 B IGKC-', 'CD4+ T activated', 'CD4+ T activated integrinB7+', ..., 'cDC2', 'dnT', 'gdT TCRVD2+', 'pDC']


In [16]:
gex_meta.var

,gene_ids,feature_types,genome,n_cells-s1d1,highly_variable-s1d1,n_cells-s1d2,highly_variable-s1d2,n_cells-s1d3,highly_variable-s1d3,n_cells-s2d1,highly_variable-s2d1,n_cells-s2d4,highly_variable-s2d4,n_cells-s3d6,highly_variable-s3d6,n_cells-s3d7,highly_variable-s3d7
AL627309.5,ENSG00000241860,GEX,GRCh38,69,False,91,False,48,False,116,False,118,True,120,False,182,False
LINC01409,ENSG00000237491,GEX,GRCh38,303,False,376,False,362,False,318,False,370,False,508,False,611,False
LINC01128,ENSG00000228794,GEX,GRCh38,703,True,406,True,584,True,780,True,547,True,663,True,563,True
LINC00115,ENSG00000225880,GEX,GRCh38,103,True,86,False,66,False,61,False,60,False,60,False,162,True
FAM41C,ENSG00000230368,GEX,GRCh38,130,False,91,False,95,False,81,False,94,False,93,False,222,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-CYB,ENSG00000198727,GEX,GRCh38,6136,False,5468,False,6667,False,12854,False,5724,False,13166,False,14143,False
AC011043.1,ENSG00000276256,GEX,GRCh38,51,False,25,False,44,True,49,False,30,False,60,False,35,False
AL592183.1,ENSG00000273748,GEX,GRCh38,482,True,174,True,153,True,334,True,356,True,238,False,821,False
AC240274.1,ENSG00000271254,GEX,GRCh38,53,False,66,False,54,False,35,False,93,False,158,False,79,False


In [23]:
counts = gex_meta.layers['counts']

In [33]:
total_counts = counts.sum(axis=1).A1
total_counts.min(), total_counts.max()

(676.0, 37118.0)

In [36]:
total_detected = (counts > 0.0).sum(axis=1).A1
total_detected.min(), total_detected.max()

(102, 5691)

In [37]:
gene_detected_in_cells = (counts > 0.0).sum(axis=0).A1
gene_detected_in_cells.min(), gene_detected_in_cells.max()

(61, 29031)

# Adt

In [41]:
adt_meta = ad.read_h5ad("data/official/common/openproblems_bmmc_cite_phase1/openproblems_bmmc_cite_phase1.manual_formatting.output_mod2.h5ad")
adt_meta = adt_meta[adt_meta.obs['is_train'], :]

In [44]:
adt_meta.obs.describe()

,n_antibodies_by_counts,total_counts,iso_count,pseudotime_order_ADT,pseudotime_order_GEX
count,29077.000000,29077.000000,29077.000000,5739.000000,5791.000000
mean,108.407126,1876.793067,7.247343,0.773514,0.738605
std,13.870463,2915.596638,5.760087,0.272619,0.299480
min,61.000000,200.000000,1.000000,0.000000,0.000000
25%,100.000000,481.000000,4.000000,0.673726,0.737430
50%,109.000000,715.000000,6.000000,0.923920,0.865717
75%,118.000000,1511.000000,9.000000,0.966033,0.939360
max,140.000000,23996.000000,78.000000,1.000000,1.000000


In [54]:
counts = adt_meta.layers['counts']

In [59]:
total_counts = counts.sum(axis=1).A1
total_counts.min(), total_counts.max()

(197.0, 23932.0)